<a href="https://colab.research.google.com/github/aninatroya/ABSA_Tweets/blob/main/Beyond_Meat_Twitter_Historical_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start here

In [ ]:
! pip install tweepy --upgrade

  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
! pip3 install snscrape

### Links:
- [Historical data twitter issue](https://www.researchgate.net/post/How-to-get-twitter-historical-data)
- https://github.com/JustAnotherArchivist/snscrape
- https://developer.twitter.com/en/docs/twitter-api/v1/tweets/post-and-engage/api-reference/get-statuses-lookup

In [ ]:
! snscrape twitter-search "#BeyondMeat since:2019-01-01 until:2019-06-15" > beyondmeet_tweets.txt

In [ ]:
import pandas as pd, tweepy

In [ ]:
#Credential variables 
ACCESS_TOKEN= '...'
ACCESS_TOKEN_SECRET = '...'
CONSUMER_KEY = '...'  
CONSUMER_SECRET = '...'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET) 
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
tweet_url = pd.read_csv("beyondmeet_tweets.txt", index_col= None, header = None, names = ["links"])
tweet_url.head()

,links
0,https://twitter.com/ChronosCap/status/11396796...
1,https://twitter.com/LenovoLegion/status/130158...
2,https://twitter.com/Celestialvegan/status/1139...
3,https://twitter.com/daswarkeinhuhn/status/1139...
4,https://twitter.com/jmartin795/status/11396675...


In [ ]:
tweet_url.shape

(5170, 1)

In [ ]:
# For every element x , select x["links"],
# then split it by ("/"), and save the last part of the split
### Definition of lambda expression
af = lambda x: x["links"].split("/")[-1]
### Application of the lmabda on the datat
tweet_url['id'] = tweet_url.apply(af, axis=1)

In [ ]:
ids = tweet_url['id'].tolist()

In [ ]:
total_count = len(ids)
chunks = (total_count - 1) // 50 + 1

In [ ]:
total_count

5170

In [ ]:
def fetch_tw(ids):
    # looking up a list of statues (Tweets) on the API
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": status.id,
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at,
                     "user_location": status.user.location, 
                     "hashtags": status.entities.hashtags}
            # You can add more attributes.....
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv("beyondmeat_tweets.csv", mode="a")

In [ ]:
for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result = fetch_tw(batch)

In [ ]:
pd.read_csv("beyondmeat_tweets.csv")

,Unnamed: 0,date,screen_name,tweet,tweet_id,user_location
0,0.0,2019-06-14 23:42:59,ChronosCap,The only thing missing from the #BeyondMeat ep...,1.1396796570563584e+18,Milky Way ✨
1,1.0,2019-06-14 20:04:15,erich_strasser,"Red Robin Investor Eyes Beyond Meat’s Gains, S...",1.1396246133343068e+18,Österreich
2,2.0,2019-06-14 16:16:54,IBD_MLarkin,#BeyondMeat Rallies As Fake Meat Rival #Imposs...,1.1395673982629888e+18,NaN
3,3.0,2019-06-14 17:20:43,PURISfoods,Ever wonder what the future of food looks like...,1.1395834576869581e+18,"Minneapolis, MN"
4,4.0,2019-06-14 16:00:40,ParagonAlert,"This year's #IPO ""unicorns"" are #BeyondMeat, #...",1.1395633119750595e+18,"Hoboken, NJ"
...,...,...,...,...,...,...
5268,15.0,2019-01-02 13:50:35,michaelwolf,Beyond Meat Cooks Up Fast Food Partnership wit...,1.080461352613761e+18,Seattle
5269,16.0,2019-01-02 18:59:33,juliedabrusin,@sofeecee @AWCanada Interesting. I was just ha...,1.0805391085760553e+18,NaN
5270,17.0,2019-01-01 21:49:30,FRANKYBANANAS,A couple hours after my run I was starving! Sh...,1.0802194875996815e+18,"Orange County, CA"
5271,18.0,2019-01-02 02:11:54,Cherrypie_9,#beyondmeat I had the best New Years last nigh...,1.0802855255613604e+18,NaN
